In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("Inputdata.csv")

In [5]:
df

,Ngày,Tiêu đề,Nguồn,Mã CK \nliên quan,URL,filename,text,argument_classification_label,sentiment_analysis_label,impact_duration_label,Unnamed: 10,updated_date
0,22/10/2025,KBC – BCPT – Mở rộng quỹ đất KCN mang đến độn...,MBS,liên quan,https://cafef1.mediacdn.vn/Images/Uploaded/DuL...,KBC_BCPT_2025102222102025153348.pdf,Chúng tôi kỳ vọng KBC bàn giao 30 ha đất tại K...,Kịch bản,Tăng,1-3 tháng,NaN,2025-12-01
1,22/10/2025,KBC – BCPT – Mở rộng quỹ đất KCN mang đến độn...,MBS,KBC,https://cafef1.mediacdn.vn/Images/Uploaded/DuL...,KBC_BCPT_2025102222102025153348.pdf,KBC liên tiếp nhận được quyết định chấp thuận ...,Kịch bản,Tiếp tục tăng trưởng,> 1 năm,NaN,2025-12-01
2,22/10/2025,KBC – BCPT – Mở rộng quỹ đất KCN mang đến độn...,MBS,KBC,https://cafef1.mediacdn.vn/Images/Uploaded/DuL...,KBC_BCPT_2025102222102025153348.pdf,Chúng tôi giữ nguyên khuyến nghị KHẢ QUAN và n...,Kết luận,Tích cực,7-12 tháng,NaN,2025-12-01
3,22/10/2025,KBC – BCPT – Mở rộng quỹ đất KCN mang đến độn...,MBS,KBC,https://cafef1.mediacdn.vn/Images/Uploaded/DuL...,KBC_BCPT_2025102222102025153348.pdf,(1) Mức thuế suất Mỹ áp cho hàng hoá Việt Nam ...,Tiền đề,Tích cực,> 1 năm,NaN,2025-12-01
4,22/10/2025,KBC – BCPT – Mở rộng quỹ đất KCN mang đến độn...,MBS,KBC,https://cafef1.mediacdn.vn/Images/Uploaded/DuL...,KBC_BCPT_2025102222102025153348.pdf,Chúng tôi khuyến nghị KHẢ QUAN cho cổ phiếu KB...,Kết luận,Tích cực,7-12 tháng,NaN,2025-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...
16970,########,BÁO CÁO KHUYẾN NGHỊ ĐẦU TƯ - MUA - CTR (Q1/2025),DSCS,CTR,https://cafef1.mediacdn.vn/Images/Uploaded/DuL...,BC_CTR_Q12025_cb59ca116c17062025152123.pdf,CTR chuyển sang niêm yết trên sàn HSX vào thán...,Tiền đề,Trung lập,> 1 năm,NaN,2025-12-01
16971,########,BÁO CÁO KHUYẾN NGHỊ ĐẦU TƯ - MUA - CTR (Q1/2025),DSCS,CTR,https://cafef1.mediacdn.vn/Images/Uploaded/DuL...,BC_CTR_Q12025_cb59ca116c17062025152123.pdf,"Vào tháng 12/2020, Viettel đã thực hiện một đợ...",Kịch bản,Không xác định,> 1 năm,NaN,2025-12-01
16972,########,BÁO CÁO KHUYẾN NGHỊ ĐẦU TƯ - MUA - CTR (Q1/2025),DSCS,CTR,https://cafef1.mediacdn.vn/Images/Uploaded/DuL...,BC_CTR_Q12025_cb59ca116c17062025152123.pdf,Các lãnh đạo cấp cao trong đội ngũ BLĐ đều có ...,Tiền đề,Tích cực,> 1 năm,NaN,2025-12-01
16973,########,BÁO CÁO KHUYẾN NGHỊ ĐẦU TƯ - MUA - CTR (Q1/2025),DSCS,CTR,https://cafef1.mediacdn.vn/Images/Uploaded/DuL...,BC_CTR_Q12025_cb59ca116c17062025152123.pdf,Chúng tôi xác nhận rằng những quan điểm được t...,Tiền đề,Trung lập,> 1 năm,NaN,2025-12-01


In [6]:
import argparse, os, json, re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import joblib

In [8]:
# train_tfidf_svm.py
"""
Train TF-IDF + LinearSVC on your CSV.
Usage:
  python train_tfidf_svm.py --csv "/path/to/your.csv" --premise_label_col "argument_classification_label" --text_col "text" --sentiment_col "sentiment_analysis_label"

Outputs:
  - /output/models/tfidf_svm_pipeline.joblib
  - /output/models/label_map.json
  - /output/models/train_phobert_train.csv, train_phobert_val.csv  (small CSVs for phobert)
"""

def clean_text(s):
    s = str(s)
    s = s.replace('\n',' ').replace('\r',' ')
    s = s.lower()
    s = re.sub(r'http\S+', ' ', s)
    s = re.sub(r'[^a-z0-9ạáàảãâấầẩẫăắằẳẵặẹéèẻẽêếềểễìíỉĩòóỏõôốồổỗơớờởỡụùúủũưứừửữỳýỷỹđ\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def main(args):
    df = pd.read_csv(args.csv, encoding='utf-8', on_bad_lines='skip')
    # Select columns
    text_col = args.text_col
    arg_col = args.premise_label_col
    sent_col = args.sentiment_col

    # Filter premise rows (exact match or contains)
    prem_df = df[df[arg_col].astype(str).str.strip() == 'Tiền đề'].copy()
    if len(prem_df) == 0:
        prem_df = df[df[arg_col].astype(str).str.lower().str.contains('tiền') & df[arg_col].astype(str).str.lower().str.contains('đề')].copy()

    prem_df = prem_df.dropna(subset=[text_col, sent_col])
    prem_df = prem_df[prem_df[text_col].astype(str).str.strip() != '']
    prem_df['text_clean'] = prem_df[text_col].apply(clean_text)
    prem_df[sent_col] = prem_df[sent_col].astype(str).str.strip()
    prem_df = prem_df[prem_df[sent_col] != '']

    X = prem_df['text_clean'].values
    y = prem_df[sent_col].values

    if len(X) < 20:
        raise SystemExit("Quá ít sample tiền đề để train. Cần ít nhất vài chục sample.")

    le = LabelEncoder()
    y_enc = le.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y_enc, test_size=0.2, random_state=42,
        stratify=y_enc if len(set(y_enc))>1 else None
    )

    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(ngram_range=(1,2), max_features=20000)),
        ('clf', LinearSVC(random_state=42, max_iter=10000))
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    print("Classification report:")
    print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0))
    print("Confusion matrix:")
    print(confusion_matrix(y_test, y_pred))

    out_dir = args.output_dir
    os.makedirs(out_dir, exist_ok=True)
    model_path = os.path.join(out_dir, "tfidf_svm_pipeline.joblib")
    label_map_path = os.path.join(out_dir, "label_map.json")
    joblib.dump(pipeline, model_path)
    with open(label_map_path, 'w', encoding='utf-8') as f:
        json.dump({"classes": list(le.classes_)}, f, ensure_ascii=False, indent=2)
    print("Saved model to:", model_path)
    print("Saved label map to:", label_map_path)

    # Save small CSVs for PhoBERT fine-tuning (text,label)
    train_df = pd.DataFrame({'text': list(X_train), 'label': [le.classes_[i] for i in y_train]})
    val_df   = pd.DataFrame({'text': list(X_test),  'label': [le.classes_[i] for i in y_test]})
    train_df.to_csv(os.path.join(out_dir, "train_phobert_train.csv"), index=False, encoding='utf-8')
    val_df.to_csv(os.path.join(out_dir, "train_phobert_val.csv"), index=False, encoding='utf-8')
    print("Wrote train/val CSVs to output directory.")

AttributeError: 'DataFrame' object has no attribute 'csv'

### TF-IDF + SVM

In [20]:
import os, re, json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import joblib

# ---- CONFIG ----
csv_path = "Inputdata.csv"      # Đổi nếu file tên khác
text_col = "text"
arg_col = "argument_classification_label"
sent_col = "sentiment_analysis_label"
output_dir = "./output/models"
os.makedirs(output_dir, exist_ok=True)

# ---- CLEANING ----
def clean_text(s):
    s = str(s)
    s = s.replace('\n',' ').replace('\r',' ')
    s = s.lower()
    s = re.sub(r'http\S+', ' ', s)
    s = re.sub(r'[^a-z0-9ạáàảãâấầẩẫăắằẳẵặẹéèẻẽêếềểễìíỉĩòóỏõôốồổỗơớờởỡụùúủũưứừửữỳýỷỹđ\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

In [21]:
# 1) đọc file
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"Không tìm thấy file {csv_path}")

df = pd.read_csv(csv_path, encoding='utf-8', on_bad_lines='skip')
print("Loaded:", df.shape)
print("Columns:", df.columns.tolist())

# 2) kiểm tra cột
for col in [text_col, arg_col, sent_col]:
    if col not in df.columns:
        raise ValueError(f"Không tồn tại cột '{col}'!")

# 3) lọc Tiền đề
prem_df = df[df[arg_col].astype(str).str.strip() == "Tiền đề"].copy()
if len(prem_df) == 0:
    prem_df = df[df[arg_col].astype(str).str.lower().str.contains("tiền")].copy()

print("Rows Tiền đề:", len(prem_df))

# 4) loại rỗng
prem_df = prem_df.dropna(subset=[text_col, sent_col])
prem_df = prem_df[prem_df[text_col].astype(str).str.strip() != ""]
print("After clean empty:", len(prem_df))

# 5) làm sạch văn bản
prem_df["text_clean"] = prem_df[text_col].apply(clean_text)

prem_df[[text_col, "text_clean", sent_col]].head(10)

Loaded: (16975, 12)
Columns: ['Ngày', 'Tiêu đề', 'Nguồn', 'Mã CK \nliên quan', 'URL', 'filename', 'text', 'argument_classification_label', 'sentiment_analysis_label', 'impact_duration_label', 'Unnamed: 10', 'updated_date']
Rows Tiền đề: 11547
After clean empty: 11547


,text,text_clean,sentiment_analysis_label
3,(1) Mức thuế suất Mỹ áp cho hàng hoá Việt Nam ...,1 mức thuế suất mỹ áp cho hàng hoá vi t nam kh...,Tích cực
5,Mức thuế suất Mỹ áp cho hàng hoá nhập khẩu từ ...,mức thuế suất mỹ áp cho hàng hoá nh p khẩu từ ...,Tích cực
6,Các dự án KCN đem lại doanh thu chính cho KBC ...,các d án kcn đem lại doanh thu chính cho kbc t...,Tiếp tục tăng trưởng
8,Dự án NOXH Nếnh và Tràng Duệ đóng góp chính và...,d án noxh nếnh và tràng du đóng góp chính vào ...,Trung lập
10,Báo cáo này được viết và phát hành bởi Khối Ng...,báo cáo này đư c viết và phát hành bởi khối ng...,Trung lập
11,Khuyến nghị đầu tư của MBS được xây dựng dựa t...,khuyến ngh đầu tư của mbs đư c xây d ng d a tr...,Trung lập
12,Được thành lập từ tháng 5 năm 2000 bởi Ngân hà...,đư c thành l p từ tháng 5 năm 2000 bởi ngân hà...,Trung lập
14,Giá thuê giàn neo cao nhờ triển vọng tích cực ...,giá thuê giàn neo cao nhờ triển v ng tích c c ...,Tích cực
15,Kỳ vọng thị trường khoan Việt Nam sôi động tro...,kỳ v ng th trường khoan vi t nam sôi đ ng tron...,Tích cực
16,Mảng dịch vụ khoan kỳ vọng sẽ tiếp tục khả qua...,mảng d ch vụ khoan kỳ v ng sẽ tiếp tục khả qua...,Tích cực


In [22]:
# X, y
X = prem_df["text_clean"].values
y = prem_df[sent_col].astype(str).str.strip().values

# encode label
le = LabelEncoder()
y_enc = le.fit_transform(y)
print("Sentiment labels:", list(le.classes_))

# split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.2, random_state=42,
    stratify=y_enc if len(set(y_enc)) > 1 else None
)

print("Train/Test:", len(X_train), "/", len(X_test))

# model pipeline
model = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), max_features=20000)),
    ("svm", LinearSVC(random_state=42, max_iter=10000))
])

print("Training SVM...")
model.fit(X_train, y_train)

# predict
y_pred = model.predict(X_test)

# report
print("\n===== Classification Report =====")
print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0))

print("\n===== Confusion Matrix =====")
print(confusion_matrix(y_test, y_pred))

# save model
model_path = f"{output_dir}/tfidf_svm_pipeline.joblib"
label_path = f"{output_dir}/label_map.json"
joblib.dump(model, model_path)
with open(label_path, "w", encoding="utf-8") as f:
    json.dump({"classes": list(le.classes_)}, f, ensure_ascii=False, indent=2)

print("\nSaved model:", model_path)
print("Saved label map:", label_path)

# sample predictions
sample = pd.DataFrame({
    "text": X_test[:10],
    "pred": [le.classes_[i] for i in y_pred[:10]],
    "true": [le.classes_[i] for i in y_test[:10]]
})
sample

Sentiment labels: ['Có những thay đổi lớn', 'Giảm', 'Gặp khó khăn', 'Không xác định', 'Suy giảm', 'Thách thách', 'Thách thức', 'Tiêu cực', 'Tiếp tục tăng trưởng', 'Trung lập', 'Tích cực', 'Tăng', 'Ổn định']
Train/Test: 9237 / 2310
Training SVM...

===== Classification Report =====
                       precision    recall  f1-score   support

Có những thay đổi lớn       0.38      0.27      0.32       103
                 Giảm       0.42      0.26      0.32        66
         Gặp khó khăn       0.40      0.12      0.18        52
       Không xác định       0.67      0.09      0.15        23
             Suy giảm       0.53      0.47      0.50        66
          Thách thách       0.00      0.00      0.00         1
           Thách thức       0.53      0.37      0.44        92
             Tiêu cực       0.51      0.45      0.48       110
 Tiếp tục tăng trưởng       0.55      0.51      0.53       156
            Trung lập       0.78      0.81      0.79       605
             Tích cực   

,text,pred,true
0,lũy kế 6 tháng đầu năm doanh thu của dxg đạt 1...,Suy giảm,Có những thay đổi lớn
1,acbs không cung cấp d ch vụ tư vấn thuế và khô...,Trung lập,Trung lập
2,mã cổ phiếu pvd tạo đáy dài hạn tháng 3 2020 t...,Tiếp tục tăng trưởng,Có những thay đổi lớn
3,thu nh p lãi thuần đạt 2 504 tỷ đồng 5 8 yoy n...,Tích cực,Gặp khó khăn
4,doanh thu suy giảm đến từ i giảm hi u suất già...,Suy giảm,Suy giảm
5,tại thời điểm th c hi n báo cáo phân tích fpts...,Trung lập,Trung lập
6,khoản đầu tư tài chính dài hạn đứng thứ 2 tron...,Tích cực,Tích cực
7,tìm hướng kinh doanh mới trong cu c h p các ch...,Tích cực,Tích cực
8,khuyến ngh đầu tư đư c đưa ra d a trên khả năn...,Trung lập,Trung lập
9,trong q4 2024 hpg ghi nh n doanh thu h p nhất ...,Tăng,Tích cực


### TF - IDF + Logistic

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Load file CSV
df = pd.read_csv("Inputdata.csv")

df.head()

,Ngày,Tiêu đề,Nguồn,Mã CK \nliên quan,URL,filename,text,argument_classification_label,sentiment_analysis_label,impact_duration_label,Unnamed: 10,updated_date
0,22/10/2025,KBC – BCPT – Mở rộng quỹ đất KCN mang đến độn...,MBS,liên quan,https://cafef1.mediacdn.vn/Images/Uploaded/DuL...,KBC_BCPT_2025102222102025153348.pdf,Chúng tôi kỳ vọng KBC bàn giao 30 ha đất tại K...,Kịch bản,Tăng,1-3 tháng,NaN,2025-12-01
1,22/10/2025,KBC – BCPT – Mở rộng quỹ đất KCN mang đến độn...,MBS,KBC,https://cafef1.mediacdn.vn/Images/Uploaded/DuL...,KBC_BCPT_2025102222102025153348.pdf,KBC liên tiếp nhận được quyết định chấp thuận ...,Kịch bản,Tiếp tục tăng trưởng,> 1 năm,NaN,2025-12-01
2,22/10/2025,KBC – BCPT – Mở rộng quỹ đất KCN mang đến độn...,MBS,KBC,https://cafef1.mediacdn.vn/Images/Uploaded/DuL...,KBC_BCPT_2025102222102025153348.pdf,Chúng tôi giữ nguyên khuyến nghị KHẢ QUAN và n...,Kết luận,Tích cực,7-12 tháng,NaN,2025-12-01
3,22/10/2025,KBC – BCPT – Mở rộng quỹ đất KCN mang đến độn...,MBS,KBC,https://cafef1.mediacdn.vn/Images/Uploaded/DuL...,KBC_BCPT_2025102222102025153348.pdf,(1) Mức thuế suất Mỹ áp cho hàng hoá Việt Nam ...,Tiền đề,Tích cực,> 1 năm,NaN,2025-12-01
4,22/10/2025,KBC – BCPT – Mở rộng quỹ đất KCN mang đến độn...,MBS,KBC,https://cafef1.mediacdn.vn/Images/Uploaded/DuL...,KBC_BCPT_2025102222102025153348.pdf,Chúng tôi khuyến nghị KHẢ QUAN cho cổ phiếu KB...,Kết luận,Tích cực,7-12 tháng,NaN,2025-12-01


In [24]:
# Chọn dữ liệu
X = df["text"].astype(str)
y = df["argument_classification_label"]

# Tách dữ liệu train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Logistic Regression
model_lr = LogisticRegression(max_iter=300)
model_lr.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=300)

In [25]:
y_pred = model_lr.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7717231222385862

Classification Report:
               precision    recall  f1-score   support

    Kết luận       0.76      0.50      0.60       453
    Kịch bản       0.56      0.40      0.47       630
     Tiền đề       0.81      0.93      0.86      2309
   Trung lập       0.00      0.00      0.00         3

    accuracy                           0.77      3395
   macro avg       0.53      0.46      0.48      3395
weighted avg       0.76      0.77      0.75      3395



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

### TF - IDF + Random Forrest

In [26]:
from sklearn.ensemble import RandomForestClassifier

# TF-IDF như trên
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train_tfidf, y_train)

RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=42)

In [27]:
y_pred_rf = rf_model.predict(X_test_tfidf)

print("Accuracy RF:", accuracy_score(y_test, y_pred_rf))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))

Accuracy RF: 0.7988217967599411

Classification Report:
               precision    recall  f1-score   support

    Kết luận       0.89      0.53      0.67       453
    Kịch bản       0.71      0.36      0.48       630
     Tiền đề       0.80      0.97      0.88      2309
   Trung lập       1.00      0.33      0.50         3

    accuracy                           0.80      3395
   macro avg       0.85      0.55      0.63      3395
weighted avg       0.80      0.80      0.77      3395

